# MPC MPECs API

#### This tutorial provides information on how to use the Minor Planet Center's MPECs API.

The Minor Planet Center's `MPECs` (Minor Planet Electronic Circulars) service returns information about MPECs associated with specific objects or search terms.

This is useful when you want to:
 - Find the discovery MPEC for an asteroid or comet
 - Look up all MPECs mentioning a specific object
 - Search for MPECs by title pattern
 - Get links to MPEC documents

The MPECs API is a REST endpoint. You can send GET requests to:

    https://data.minorplanetcenter.net/api/mpecs

In the examples below we use Python code to query the API.

Further information and documentation can be found at:
 - https://minorplanetcenter.net/mpcops/documentation/mpecs-api/
 - https://minorplanetcenter.net/mpec/RecentMPECs.html (Recent MPECs)

# Import Packages
Here we import the standard Python packages needed to call the API and interpret the returned data.

In [ ]:
import requests
import json

# API Parameters

The MPECs API accepts a JSON array of search terms:

| Input | Description |
|-------|-------------|
| JSON array | List of search terms (up to 100 terms, max 1000 results per term) |

### Search Term Types

Search terms can be:
1. **Object designations** - Any identifier resolvable by the Designation Identifier API (e.g., "Ceres", "1992 BB", "6564")
2. **MPEC names** - Packed or unpacked (e.g., "1993-S13" or "J93S13")
3. **Wildcard searches** - Using `%` for pattern matching (e.g., "2024%")

### Response Fields

Each MPEC entry includes:
- `fullname`: MPEC identifier (e.g., "2022-A05")
- `title`: MPEC title
- `pubdate`: UTC publication date
- `link`: URL to the MPEC document

# Query by Object Name

Here we search for MPECs associated with asteroid `Apophis`, one of the most studied potentially hazardous asteroids.

In [ ]:
# Search for MPECs mentioning Apophis
response = requests.get(
    "https://data.minorplanetcenter.net/api/mpecs",
    json=["Apophis"]
)

if response.ok:
    result = response.json()
    
    # Results are keyed by search term
    mpecs = result.get("Apophis", [])
    print(f"Found {len(mpecs)} MPECs for Apophis")
    print("\nFirst 5 MPECs:")
    for mpec in mpecs[:5]:
        print(f"  {mpec['fullname']}: {mpec['title'][:50]}...")
else:
    print(f"Error: {response.status_code}")

# MPEC Details

Let's examine the full details of MPEC entries.

In [ ]:
# Get MPECs for a specific object
response = requests.get(
    "https://data.minorplanetcenter.net/api/mpecs",
    json=["2023 BU"]  # Famous close-approach asteroid
)

if response.ok:
    result = response.json()
    mpecs = result.get("2023 BU", [])
    
    print(f"MPECs for 2023 BU:\n")
    for mpec in mpecs:
        print(f"MPEC: {mpec['fullname']}")
        print(f"  Title: {mpec['title']}")
        print(f"  Published: {mpec['pubdate']}")
        print(f"  Link: {mpec['link']}")
        print()
else:
    print(f"Error: {response.status_code}")

# Multiple Search Terms

You can search for multiple objects in a single API call.

In [ ]:
# Search for multiple objects
search_terms = ["Sedna", "Bennu", "`Oumuamua"]

response = requests.get(
    "https://data.minorplanetcenter.net/api/mpecs",
    json=search_terms
)

if response.ok:
    result = response.json()
    
    for term in search_terms:
        mpecs = result.get(term, [])
        print(f"{term}: {len(mpecs)} MPECs")
        if mpecs:
            # Show the most recent MPEC
            recent = mpecs[0]
            print(f"  Most recent: {recent['fullname']} - {recent['title'][:40]}...")
        print()
else:
    print(f"Error: {response.status_code}")

# Wildcard Searches

Use `%` as a wildcard to search MPEC titles. This is useful for finding all MPECs from a specific year or with certain patterns.

In [ ]:
# Search for MPECs with titles containing "2024 B"
search_str="2024 B%"

# Note: This will return many results
response = requests.get(
    "https://data.minorplanetcenter.net/api/mpecs",
    json=[search_str]  # MPECs for objects with designations starting with "2024 B"
)

if response.ok:
    result = response.json()
    mpecs = result.get(search_str, [])
    
    print(f"Found {len(mpecs)} MPECs matching '{search_str}'")
    print("\nFirst 10:")
    for mpec in mpecs[:10]:
        print(f"  {mpec['fullname']}: {mpec['title'][:50]}")
else:
    print(f"Error: {response.status_code}")

# Query by MPEC Name

You can query for a specific MPEC by its identifier.

In [ ]:
# Query specific MPECs by their identifiers
mpec_ids = ["2017-U181", "2017-U183"]  # `Oumuamua discovery and follow-up

response = requests.get(
    "https://data.minorplanetcenter.net/api/mpecs",
    json=mpec_ids
)

if response.ok:
    result = response.json()
    
    for mpec_id in mpec_ids:
        mpecs = result.get(mpec_id, [])
        if mpecs:
            mpec = mpecs[0]
            print(f"MPEC {mpec_id}:")
            print(f"  Title: {mpec['title']}")
            print(f"  Published: {mpec['pubdate']}")
            print(f"  Link: {mpec['link']}")
            print()
        else:
            print(f"MPEC {mpec_id}: Not found")
else:
    print(f"Error: {response.status_code}")

# Using Packed Designations

The API accepts packed designations as well as unpacked ones.

In [ ]:
# Compare packed vs unpacked designation queries
# K14A00A is the packed form of 2014 AA
search_terms = ["K14A00A", "2014 AA"]

response = requests.get(
    "https://data.minorplanetcenter.net/api/mpecs",
    json=search_terms
)

if response.ok:
    result = response.json()
    
    for term in search_terms:
        mpecs = result.get(term, [])
        print(f"'{term}': {len(mpecs)} MPECs found")
else:
    print(f"Error: {response.status_code}")

# Helper Functions

Here are convenient helper functions for working with the MPECs API.

In [ ]:
def get_mpecs(search_term):
    """
    Get MPECs for a single search term.
    
    Parameters
    ----------
    search_term : str
        Object designation, MPEC name, or wildcard pattern
    
    Returns
    -------
    list
        List of MPEC dictionaries
    """
    response = requests.get(
        "https://data.minorplanetcenter.net/api/mpecs",
        json=[search_term]
    )
    
    if not response.ok:
        return []
    
    return response.json().get(search_term, [])


def get_discovery_mpec(designation):
    """
    Get the discovery MPEC for an object.
    
    The discovery MPEC is typically the oldest MPEC for an object.
    
    Parameters
    ----------
    designation : str
        Object designation
    
    Returns
    -------
    dict or None
        Discovery MPEC info, or None if not found
    """
    mpecs = get_mpecs(designation)
    if not mpecs:
        return None
    
    # Sort by publication date to find the earliest
    sorted_mpecs = sorted(mpecs, key=lambda x: x.get('pubdate', ''))
    return sorted_mpecs[0] if sorted_mpecs else None


def get_mpec_url(designation):
    """
    Get the URL of the discovery MPEC for an object.
    
    Parameters
    ----------
    designation : str
        Object designation
    
    Returns
    -------
    str or None
        URL to the discovery MPEC
    """
    mpec = get_discovery_mpec(designation)
    return mpec.get('link') if mpec else None


# Example usage
print("Discovery MPEC for Bennu:")
discovery = get_discovery_mpec("Bennu")
if discovery:
    print(f"  MPEC: {discovery['fullname']}")
    print(f"  Title: {discovery['title']}")
    print(f"  Date: {discovery['pubdate']}")
    print(f"  URL: {discovery['link']}")
else:
    print("  Not found")

# Finding Recent MPECs

Use wildcard searches to find recent MPECs. Note that this approach has limitations since it searches titles, not dates.

In [ ]:
# Search for recent C-type comet MPECs from 2025 
# NB: This searches titles containing "Comet C/2025"
search_str = "%Comet C/2025%"
response = requests.get(
    "https://data.minorplanetcenter.net/api/mpecs",
    json=[search_str]
)

if response.ok:
    result = response.json()
    mpecs = result.get(search_str, [])
    
    # Sort by publication date (most recent first)
    sorted_mpecs = sorted(mpecs, key=lambda x: x.get('pubdate', ''), reverse=True)
    tmp = len(mpecs) if len(mpecs) < 1000 else "> 1,000"
    print(f"Found {tmp} MPECs containing {search_str}")
    print("\nMost recent 5:")
    for mpec in sorted_mpecs[:5]:
        print(f"  {mpec['pubdate'][:10]}: {mpec['fullname']} - {mpec['title'][:40]}...")
else:
    print(f"Error: {response.status_code}")

# Using the `mpc_api` Package

The [`mpc_api`](https://github.com/Smithsonian/mpc-public/tree/main/mpc_api) Python package wraps all MPC API calls into a single `MPCClient` class.

```bash
pip install mpc-api
```

In [ ]:
from mpc_api import MPCClient

mpc = MPCClient()

# Search for MPECs
result = mpc.get_mpecs("Apophis")
print(f"Found {len(result.get('Apophis', []))} MPECs for Apophis")

# Get the discovery MPEC
discovery = mpc.get_discovery_mpec("Bennu")
if discovery:
    print(f"Discovery MPEC: {discovery['fullname']} - {discovery['title']}")

# Summary

The MPC MPECs API provides access to Minor Planet Electronic Circulars.

Key points:
- **Endpoint**: `https://data.minorplanetcenter.net/api/mpecs`
- **Input**: JSON array of search terms
- **Search types**: Object designations, MPEC names, wildcard patterns (`%`)
- **Limits**: Up to 100 search terms, 1000 results per term
- **Response fields**: `fullname`, `title`, `pubdate`, `link`

Use cases:
- Find discovery announcements for objects
- Track MPEC history for an asteroid or comet
- Search for MPECs by pattern

For questions or feedback, contact the MPC via the [Jira Helpdesk](https://mpc-service.atlassian.net/servicedesk/customer/portal/13/create/148).